<a href="https://colab.research.google.com/github/arkwith7/aSSIST_ML/blob/main/PEFT_LoRA_token_classification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA token classification 예제 - BioNLP2004 데이터셋
## Reference : https://huggingface.co/docs/peft/task_guides/token-classification-lora

# 필요한 라이브러리 설치

In [1]:
!pip install -q peft transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:0

# 설정값 지정

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

#model_checkpoint = "roberta-large"
model_checkpoint = "roberta-base"   # 빠른 학습을 위해 base 모델로 설정
lr = 1e-3
batch_size = 16
#num_epochs = 10
num_epochs = 1      # 빠른 학습을 위해 epoch을 1로 설정

# BioNLP2004 데이터셋 불러오기
## (DNA, RNA, proteins 같은 생물학적 구조에 대한 tag를 나타내는 데이터셋입니다.)

In [3]:
# tags는 label id를 나타냅니다.
bionlp = load_dataset("tner/bionlp2004")
bionlp["train"][0]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{'tokens': ['Since',
  'HUVECs',
  'released',
  'superoxide',
  'anions',
  'in',
  'response',
  'to',
  'TNF',
  ',',
  'and',
  'H2O2',
  'induces',
  'VCAM-1',
  ',',
  'PDTC',
  'may',
  'act',
  'as',
  'a',
  'radical',
  'scavenger',
  '.'],
 'tags': [0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [4]:
# precision, accuracy, F1, and recall 등 sequence labeling tasks evaluation을 위한 모듈
seqeval = evaluate.load("seqeval")

In [5]:
label_list = [
    "O",
    "B-DNA",
    "I-DNA",
    "B-protein",
    "I-protein",
    "B-cell_type",
    "I-cell_type",
    "B-cell_line",
    "I-cell_line",
    "B-RNA",
    "I-RNA",
]

In [6]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Tokenizer 불러오기

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 토크나이징

In [9]:
tokenized_bionlp = bionlp.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/16619 [00:00<?, ? examples/s]

Map:   0%|          | 0/1927 [00:00<?, ? examples/s]

Map:   0%|          | 0/3856 [00:00<?, ? examples/s]

In [10]:
# 가장 긴 길이의 데이터에 맞게 padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [11]:
id2label = {
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-protein",
    4: "I-protein",
    5: "B-cell_type",
    6: "I-cell_type",
    7: "B-cell_line",
    8: "I-cell_line",
    9: "B-RNA",
    10: "I-RNA",
}
label2id = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# PEFT 모델 설정

In [12]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
)

# LoRA 기법으로 인해 전체 모델의 0.55%의 파라미터만 Fine-Tuning에 사용

In [13]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 708,886 || all params: 124,661,782 || trainable%: 0.5686474143294374


# Training config 설정

In [14]:
training_args = TrainingArguments(
    #output_dir="roberta-large-lora-token-classification",
    output_dir="roberta-base-lora-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Training 시작

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bionlp["train"],
    eval_dataset=tokenized_bionlp["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.207900,0.176821,0.733760,0.785161,0.758591,0.939576


TrainOutput(global_step=1039, training_loss=0.2604874320851712, metrics={'train_runtime': 182.1594, 'train_samples_per_second': 91.233, 'train_steps_per_second': 5.704, 'total_flos': 661020489129132.0, 'train_loss': 0.2604874320851712, 'epoch': 1.0})

# 학습결과 zip 파일로 압축후 다운로드

In [16]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
folder_name = "roberta-base-lora-token-classification"

# 생성될 ZIP 파일 이름
zip_file_name = "roberta-base-lora-token-classification.zip"

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 학습결과 불러오기

### BioNLP2004 데이터셋에 1epoch Fine-Tuning 완료된 zip 파일 :
roberta-base-lora-token-classification.zip https://drive.google.com/file/d/1_mEcACHQkKcTEFpOcbSjEaPgDRBasGb0/view?usp=sharing


In [17]:
# 직접 학습하지않고 학습이 완료된 zip 파일을 업로드해서 사용하고 싶은 경우에 주석을 해제해서 사용하세요!

# import zipfile

# # 압축 해제할 ZIP 파일 이름
# zip_file_name = "roberta-base-lora-token-classification.zip"

# # 압축을 해제할 대상 폴더. 이 예시에서는 같은 이름의 폴더에 압축을 해제합니다.
# extract_folder_name = "./roberta-base-lora-token-classification"

# # ZIP 파일 압축 해제
# with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder_name)

# Inference를 위해 학습이 끝난 모델 Load하기

In [18]:
#peft_model_id = "stevhliu/roberta-large-lora-token-classification"
peft_model_id = "./roberta-base-lora-token-classification/checkpoint-1039"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForTokenClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=11, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, peft_model_id)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# sample text에 대한 Inference

In [19]:
text = "The activation of IL-2 gene expression and NF-kappa B through CD28 requires reactive oxygen production by 5-lipoxygenase."
inputs = tokenizer(text, return_tensors="pt")

In [20]:
with torch.no_grad():
    logits = model(**inputs).logits

tokens = inputs.tokens()
predictions = torch.argmax(logits, dim=2)

for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('<s>', 'O')
('The', 'O')
('Ġactivation', 'O')
('Ġof', 'O')
('ĠIL', 'B-DNA')
('-', 'I-DNA')
('2', 'I-DNA')
('Ġgene', 'I-DNA')
('Ġexpression', 'O')
('Ġand', 'O')
('ĠNF', 'B-protein')
('-', 'I-protein')
('k', 'I-protein')
('appa', 'I-protein')
('ĠB', 'I-protein')
('Ġthrough', 'O')
('ĠCD', 'B-protein')
('28', 'I-protein')
('Ġrequires', 'O')
('Ġreactive', 'O')
('Ġoxygen', 'O')
('Ġproduction', 'O')
('Ġby', 'O')
('Ġ5', 'B-protein')
('-', 'I-protein')
('lip', 'I-protein')
('oxy', 'I-protein')
('gen', 'I-protein')
('ase', 'I-protein')
('.', 'O')
('</s>', 'O')
